I used the Reddit API for this project. 

### Data Prep and Cleaning

In [ ]:
import csv
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [ ]:
comments_df = pd.read_csv('./comments.csv')
submissions_df = pd.read_csv('./submissions.csv')

Now we are going to take a brief look at the raw data from the api of both the comments and submissions to see if any cleaning needs to be performed

In [ ]:
submissions_df.head(10)

In [ ]:
submissions_df.shape

In [ ]:
comments_df.head(10)

In [ ]:
comments_df.shape

As part of the collection process, if author's name's were missing, we added name = None in case the author's name had been removed from the post in order to prevent error's in collecting data. So we can now check how many missing author's names from both the submissions and the comments are within the data. 

In [ ]:
submissions_df.isnull().sum()

In [ ]:
sub_df = submissions_df.dropna()

In [ ]:
sub_df.isnull().sum()

In [ ]:
comments_df.isnull().sum()

In [ ]:
com_df = comments_df.dropna()

In [ ]:
com_df.isnull().sum()

Because it also appears the column "tot_awards_received" and "downs" may all have the same value of 0, let's check to see if that is true. If so, we can "clean" our data by removing these columns.

In [ ]:
com_df[com_df['tot_awards_received'] != 0]

In [ ]:
com_df[com_df['downs'] != 0]

Also, let's check if the "ups" value is always equal to the "score" value. If so, than we can remove the "ups" column as it is just giving the same value as the "score" value.

In [ ]:
com_df[com_df['ups'] != com_df['score']]

In [ ]:
new_com = com_df.drop(['tot_awards_received','downs',"ups"], axis=1) 
new_com.head()

### Data Characterization, Summarization, and Analysis

Now that we have cleaned up some of the comments data we had been working with, we can take a premelinear look into the network of the comments. We can set up our network with the nodes being defined by the "authors" of a post and the "parent_id" as whom the node it is linked. 

Useing the edge list function from networkx allows for the creation of the nodes and the edges at the same time by defining the source and the target. We can also make it a directed graph by applying the nx.DiGraph

In [ ]:
g = nx.from_pandas_edgelist(new_com, source="author", target="parent_id", create_using=nx.DiGraph)

We can now do some analyis by creating a new dataframe that can measure a few basic network dimensions of our graph, such as the degree and differing centrality measures using networkx. These measures, such as degree or degree centrality, are used within social network analysis to understand the different levels of "social" or "network" capital that individuals may have, due to where they "sit" within the network and with whom different individuals may be connected to. More information can be found in Networks(Newman, 2018). 

In [ ]:
net_com_df = pd.DataFrame(dict(
    DEGREE = dict(g.degree),
    DEGREE_CENTRALITY = nx.degree_centrality(g),
    EIGENVECTOR = nx.eigenvector_centrality(g),
    KATZ = nx.katz_centrality_numpy(g),
    CLOSENESS_CENTRALITY = nx.closeness_centrality(g),
)) 

In [ ]:
net_com_df.sort_values("DEGREE", ascending=False)

We can take a preliminary look at some of the statitics of the network data

In [ ]:
net_com_df.describe()

As we can see from looking at the max and not just the mean, but the 75% percentile as well, this composition of data is not normal, but highly skewed.

Let's take a closer look visually at the distribution of the network

In [ ]:
plt.hist([v for k,v in nx.degree(g)])
plt.xlabel("Degree", fontsize=15)
plt.ylabel("Amount of Authors", fontsize=15)
plt.title("Degree Distribution", fontsize=14)

In [ ]:
plt.hist(nx.centrality.degree_centrality(g).values())
plt.xlabel("Degree Centrality", fontsize=15)
plt.ylabel("Amount of Authors", fontsize=15)
plt.title("Degree Centrality Distribution", fontsize=14)

In [ ]:
plt.hist(nx.centrality.closeness_centrality(g).values())
plt.xlabel("Closeness Centrality", fontsize=15)
plt.ylabel("Amount of Authors", fontsize=15)
plt.title("Closeness Centrality Distribution", fontsize=14)

So it appears the comments network is super skewed with most people having a low degree, meaning that most people are not "well connected" and do not comment on lots of other peoples posts, but a small few having a high degree, meaning that there a few individuals that are better connected within this comments network and comment on a variety of different comments more frequently. 

And it also appears that this network is not connected, as the closeness centrality is very low for almost all the nodes. This makes sense as this would require a person or group of people to attempt to comment on submissions on almost every topic in UCD Reddit, which could take a lot of effort for a subreddit. 

Let's take a look at visually what the network looks like, again with the nodes being the authors of a comment and the edges being whom the comment was directed towards. 

In [ ]:
fig = plt.figure(figsize=(20,20))
nx.draw(g, node_size=30)
plt.axis('equal')
plt.show()

Wow, this network is quite large. Drawing this graph via nx.draw allows for us to visually see the network. Networkx places those nodes with the greater amount of connections focused in the inside, with those least connected on the outside. As we had already seen looking at the degree and the centrality distributions, the network is quite large and quite skewed, following similar distributions in many real world social networks. 

Now let's shift our focus at the submission network that represents those who have posted a topic on the UCD reddit. 

In [ ]:
s = nx.from_pandas_edgelist(sub_df, source="author", target="id", create_using=nx.DiGraph)

In [ ]:
net_sub_df = pd.DataFrame(dict(
    DEGREE = dict(s.degree),
    DEGREE_CENTRALITY = nx.degree_centrality(s),
    EIGENVECTOR = nx.eigenvector_centrality(s, max_iter=600),##here we use the max_iteration argument as it's default value is 100, which for this measure fails to converge
    KATZ = nx.katz_centrality_numpy(s),
    CLOSENESS_CENTRALITY = nx.closeness_centrality(s),
)) 

In [ ]:
net_sub_df.sort_values("DEGREE", ascending=False)

In [ ]:
net_sub_df.describe()

As we can see, the difference between the mean and the max of submission data is less drastic than the comment data, but still appears to be skewed. However, we can take a look again at a few of the same distributions.

In [ ]:
plt.hist([v for k,v in nx.degree(s)])
plt.xlabel("Degree", fontsize=15)
plt.ylabel("Amount of Authors", fontsize=15)
plt.title("Degree Distribution", fontsize=14)

In [ ]:
plt.hist(nx.centrality.degree_centrality(s).values())
plt.xlabel("Degree Centrality", fontsize=15)
plt.ylabel("Amount of Authors", fontsize=15)
plt.title("Degree Centrality Distribution", fontsize=14)

Now let's visualize what the network looks like

In [ ]:
fig = plt.figure(figsize=(20,20))
nx.draw(s, node_size=30)
plt.axis('equal')
plt.show()

With this many nodes and edges within the data, especially the comments data, networkx may not be the best way to visualize the network. The Gephi application may be a better tool as it can allow for us to better handle large networks and use the time data we have stored within our dataframes, creating a dynamic graph (which networkx does not allow for). It can also better represent bipartite networks to create a better visual representation, as this may be a better way to represent the the submission network, as the submission network nodes can both represents those who author a submission, and the submission itself.

Thus, we can write out both our comments data and our submissions data to csv files. As csv is one of the two types of file that works best for loading data into Gephi, this can be our best option. Importantly, we are still keeping in the text for both the submission and the comments, in case one wants to also perform discourse network analysis on the data, this will keep what was actually said in UCD Subreddit.

In [ ]:
new_com.to_csv('./cleaned_comments_data.csv', index=False)

In [ ]:
sub_df.to_csv('./cleaned_submission_data.csv', index=False)

However, our journey is not over yet. We still can take a look at the data from comments and the submissions and see what we may gleam by looking into the number of posts, the number of comments, and the "amount they had to say" in the comments of those in UCD reddit by merging our two cleaned dataframes. To understand the "amount they had to say", we can count the amount of strings within the text of the comment body and add that as a new variable.

In [ ]:
new_com['comment_length'] = new_com.body.str.len()

We can use the groupby function to collect both the comment data and the submission data. For the comment data, we want to include the number of comments, the average comment length per person, and the average score the person would recieve. For the submissions data, as we do not have a comment length or a score, we can group by the number of posts that someone made.

In [ ]:
commenter_stats = new_com.groupby('author').agg(
    num_comments = ('id', 'count'),
    median_comment_length = ('comment_length', 'median'),
    median_score = ('score', 'median'),
).reset_index()

In [ ]:
submitter_stats = sub_df.groupby('author').agg(
    num_posts = ('id', 'count')
).reset_index()

Now we can merge these two based upon the author.

In [ ]:
sub_com = pd.merge(commenter_stats, submitter_stats, on='author')

now we can take a look at the distribution of the merged data, including the amount of posts and the amount of comments

In [ ]:
plt.hist(sub_com['num_posts'])
plt.xlabel("Amount of Posts", fontsize=15)
plt.ylabel("Amount of Authors", fontsize=15)
plt.title("Posts within UCD Reddit", fontsize=14)

In [ ]:
plt.hist(sub_com['num_comments'])
plt.xlabel("Amount of Comments", fontsize=15)
plt.ylabel("Amount of Authors", fontsize=15)
plt.title("Comments within UCD Reddit", fontsize=14)

The amount of comments and posts appear to be similar in distribution to many small world, scale free networks that are common in many social networks. While this may not be surprising that UCD Redditt has similar properties to other small world, scale free networks, it is important to note that this does match with our earlier data in which we saw similar distributions within the networks of both the comment data and the submissions data. 

We have not taken a look at the the distribution of the comment length. Let's take a look at that before we dive further into our analysis.

In [ ]:
plt.hist(sub_com['median_comment_length'])
plt.xlabel("Comment Length of Posts", fontsize=15)
plt.ylabel("Amount of Authors", fontsize=15)
plt.title("Post's within UCD Reddit", fontsize=14)

One question that we can ask is if someone comments longer, does this mean that they may be more likely to comment on other posts? Are they more comfortable in writing a lot and so are more comfortable to make more comments? Because the comment length and the number of comments are both so skewed, we can take the log of the length of comments, as well as the number of comments to see if there is a relationship between the two. 

While we do this, we can create a regression line to see what the relation between the comment length and the number of comments someone has.

In [ ]:
model = LinearRegression()

X = np.log(sub_com[['median_comment_length']].values)
Y = np.log(sub_com[['num_comments']].values)
M = model.fit(X, Y)

In [ ]:
plt.figure(figsize=(8,6) )
# set up the scatter plot
p = plt.scatter(X, Y, s=40)
plt.plot(X, model.predict(X), color="red")
plt.xlabel("Log of the Comment Length of Posts", fontsize=15)
plt.ylabel("Log of the Number of Comments", fontsize=15)
plt.title("Post's within UCD Reddit", fontsize=14)

It appears that there is very little relationship between how often someone comments and the length of the posts. This linear model suggest that the longer the comments, on average the more often this person will comments in the UCD Reddit...but only marginally. 

However, we did not split our data to train the model. This model then may not be the most conclusive, may not represent the relationship, and can be overfit to our data. 

Thus, to get further insigths, instead of taking the log of our values, we can also normalize our data, as we have seen in 18-Intro to Classification, which will include both the comment length and the score to see the relation between the two variables.

We can also use the linear regression from sklearn that was used in 17-Linear Regression, as we did previously, but this time with the data split via a training and testing split.

In [ ]:
data = sub_com[["num_comments",'median_comment_length',"median_score","num_posts"]]

In [ ]:
normalizer = MinMaxScaler().set_output(transform = "pandas")
data_scaled = normalizer.fit_transform(data)
data_scaled

Before we start to build our model, we can to a preliminary check to see how the data is correlated. We can do this to check for the relationship between the two variable of comment length and number of comments. 

In [ ]:
data_scaled.corr()

There appears to be little corelation between the comment length and the number of comments.

However, as we can see there is a slight correlation between median_comment_length and median score. We can create a model to try to predict the score based off of the comment length. 

Thus, we can switch our attention to creating a model for predicting the median score based off of the median comment length. 

Here we will train a linear regression model on 60% of our data and save 40% for testing.

In [ ]:
x = data_scaled[["median_comment_length"]].values
y = data_scaled[["median_score"]]
data_train, data_test, target_train, target_test = train_test_split(x, y, test_size = 0.4)
print("Training set has %d examples" % data_train.shape[0])
print("Test set has %d examples" % data_test.shape[0])

In [ ]:
model = LinearRegression()
mod = model.fit(data_train, target_train)

Let's take a look at the slope of the model and see if there is a strong positive or negative correlation both

In [ ]:
print("Model intercept is %.4f" % model.intercept_)
print("Model slope is %.4f" % model.coef_[0])

Let's test how good our linear model is by finding the mean squared error

In [ ]:
predictions = model.predict(data_test)
predictions

In [ ]:
mse = mean_squared_error(target_test, predictions)
print("MSE=%.2f" % mse)

As we have a low mean squared error, this model appears to be a good fit...too good of a fit and may be overfit for our data. 

However, let's take a look at what this relationship looks like visually by adding in the model's linear regression line to our data.

In [ ]:
X_fit = np.linspace(0, 1, 100)[:, np.newaxis]
y_fit = model.predict(X_fit)
plt.figure(figsize=(8, 6))
plt.plot(x, y, 'o')
plt.plot(X_fit, y_fit)
plt.xlabel("median coment length per author", fontsize=14)
plt.ylabel("median score per author", fontsize=14);

As we can see, there is not that strong of a drastic positive relationship between the amount someone writes in a comments and the score that they will recieve. Maybe it will go up slightly by the amount someone writes, but, as one can easily imagine, a person cannot just increase their comment length drasticaly to get a drastic better score/ie the more upvotes one can recieve with the least downvotes on a particular comment.

### Discussion

In completing this project, there were a few challenges that were difficult to face, both in the colllection of the data via the Reddit api and in the cleaning and analysis of the data. 

I wanted to learn how to use wrappers when accessing API's, and so I chose to use the PRAW wrapper. However, one of the problem I had with using this wrapper was retrieving the data as a list. The PRAW wrapper returns the data from the api call for comments as lists instead of in a json format with objects or arrays for all the comments in the "comment forrest", and as I was most interested in looking into comments and the social network analysis of the UCD Subreddit, I chose to just use this method of lists and wrote it into a csv. This allowed me to gain all the comments for each of the submissions to a particular topic. But I think it probably would have been better if it was returned in json format, however, I struggled to figure out how to do that with the PRAW wrapper. 

Another problem I faced was in making sure to create the "save game" function or a "starting point" when I would get an error message of too many calls a portion of the way through making calls to the Reddit Api. By just adding in the command to convert from false to true and to require the collection of only collecting from submissions that have the value of false, when I got an error message, I could re-run the cell and not re-collect all the previous data, but instead continue on with "where I had left off". 

In the cleaning and analysis portion, I knew I wanted to do a bit of social network analysis, however I have not used the networkx in python before. This was a bit of a challenge to learn and it did not seem to best visually represent networks, such as a program like Gephi can. I struggled with making an image of the network that could visually display some of the dynamics that were at play, such as changing the size of the node based on it's degree. If I did this, then the network image had a few rather large nodes, and too many tiny nodes to place. The edges in the image of the network were also too large and made it difficult to see the different connections between the nodes. 

However, using Networkx was quite helpful in displaying some of the distribution of the networks. They both followed many real world social networks, and so finding that their were a few individuals who had a greater amount of degree or degree centrality was beneficial. This is quite important as it can give some key indicators in which individuals had this amount of degree centrality and that these indivuals may hold greater amount of influence within within the UCD Subreddit. This is quite important in future study, as will be discussed below. Also, being able to perform linear regression tests on the data between the length of the comment and the score was beneficial because as a social scientist, it would not hold too much weight in different aspects of social theory if the relationships in the network were only determined by how many words someone could write. Importantly, it was beneficial to see that there was not a strong correlation in general between the amount of comments, the length of the comments, the amount of posts, and the score. This allows for the future study of the network to relate more towards discourse network analysis. In this way, the data collected and already analyzed can play a vital role in addressing which content of which actors may be best to spend more time in the qualitative analysis of the discourse, while at the same time giving some of the quantitative data of the centrality measures to the future researchers. 

In the methodology of discourse network analyis, a simplified approach is one in which a person applies both discourse analysis to the content of a text as well as network analysis who made the content and whom it was directed towards. If this approach is taken, both the comments and submission text are preserved in the cleaned data that was saved as a csv. One component that should be mentioned to the further researchers is that, when getting the data from the Reddit API, I requested in my call to include the upvotes, the downvotes, the total amount of awards, and the overall score of a comment. These are all indicators of social reward/approval or punishment/disapproval. This could have been quite supportive and better indicators of how comments were viewed, instead of just an overall score. However, I noticed when beginning the cleaning processes that the downvotes and the total amount of rewards were all 0. Also, the upvotes had negative values and perfectly matched the score, and so I believe that it may be hard to get a social signal from a comment; if a particuler comment may have gotten 12 upvotes and 10 downvotes, it could have only a score of 2, but been an important comment that recieved both positive and negative views from others. Thus, when future researchers may be using the cleaned data, the measurement value of the score may is an aggregate score of both the positive and negative ratings and this should be taken into consideration. 

